**Make sure you upload the files using this code snippet so they get sent to the correct folder, you could either upload the database from the scripts or put the CSV's here. If you upload the database go straight to task 3**

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving interval.csv to interval.csv
Saving robot.csv to robot.csv
Saving t5.csv to t5.csv
Saving t4.csv to t4.csv
Saving t3.csv to t3.csv
Saving t2.csv to t2.csv
Saving t1.csv to t1.csv


In [ ]:
# Script 1: create_schema.py
# --------------------------
# Creates the SQLite database and tables according to the final schema.

import sqlite3
import os

DB_FILE = "robot.db"

# Remove existing database if present for a clean setup
if os.path.exists(DB_FILE):
    os.remove(DB_FILE)

conn = sqlite3.connect(DB_FILE)
cursor = conn.cursor()

# Enable foreign key enforcement
cursor.execute("PRAGMA foreign_keys = ON;")

# Create Robot table
cursor.execute("""
CREATE TABLE Robot (
  robot_id INTEGER PRIMARY KEY,
  name     TEXT    NOT NULL
);
""")

# Create TargetInterval table
cursor.execute("""
CREATE TABLE TargetInterval (
  interval_id    INTEGER PRIMARY KEY AUTOINCREMENT,
  start_time_sec INTEGER NOT NULL,
  end_time_sec   INTEGER NOT NULL,
  event_type     TEXT    NOT NULL
);
""")

# Create SensorReading table, folding in both 1:N relationships:
# robot_id   FK → Robot        (HasReading)
# interval_id FK → TargetInterval (OccursDuring, nullable)
cursor.execute("""
CREATE TABLE SensorReading (
  robot_id    INTEGER NOT NULL,
  timestamp   INTEGER NOT NULL,
  x_cm        REAL    NOT NULL,
  y_cm        REAL    NOT NULL,
  interval_id INTEGER,
  PRIMARY KEY (robot_id, timestamp),
  FOREIGN KEY (robot_id)
    REFERENCES Robot(robot_id),
  FOREIGN KEY (interval_id)
    REFERENCES TargetInterval(interval_id)
);
""")

conn.commit()
conn.close()

print("Schema created successfully in 'robot.db'.")

Schema created successfully in 'robot.db'.


In [ ]:
import sqlite3
import csv
import os

DB_FILE = "robot.db"
ROBOT_CSV = "robot.csv"
INTERVAL_CSV = "interval.csv"
TRAJ_FILES = ["t1.csv", "t2.csv", "t3.csv", "t4.csv", "t5.csv"]

def load_robots(conn):
    with open(ROBOT_CSV, newline="") as f:
        reader = csv.reader(f)
        for row in reader:
            robot_id, name = row
            conn.execute(
                "INSERT INTO Robot(robot_id, name) VALUES (?, ?)",
                (int(robot_id), name)
            )
    conn.commit()
    print("Loaded robots.")

def load_intervals(conn):
    with open(INTERVAL_CSV, newline="") as f:
        reader = csv.reader(f)
        for row in reader:
            start_sec, end_sec, evt = row
            conn.execute(
                "INSERT INTO TargetInterval(start_time_sec, end_time_sec, event_type) VALUES (?, ?, ?)",
                (int(start_sec), int(end_sec), evt)
            )

    conn.commit()
    print("Loaded target intervals.")

def load_trajectory(conn, traj_file, robot_id):
    with open(traj_file, newline="") as f:
        reader = csv.reader(f)
        for i, (x_str, y_str) in enumerate(reader, start=1):
            # DEBUG print(f"Timestamp: {i}")
            cursor = conn.execute("""
                SELECT interval_id
                FROM TargetInterval
                WHERE ? BETWEEN start_time_sec AND end_time_sec
            """, (i,))
            interval = cursor.fetchone()
            interval_id = interval[0] if interval else None

            conn.execute(
                "INSERT INTO SensorReading(robot_id, timestamp, x_cm, y_cm, interval_id) VALUES (?, ?, ?, ?, ?)",
                (robot_id, i, float(x_str), float(y_str), interval_id)
            )
    conn.commit()
    print(f"Loaded {traj_file} → Robot {robot_id}")

#Manually run the loading process below
if not os.path.exists(DB_FILE):
    raise SystemExit(f"Error: database file '{DB_FILE}' not found. Upload it first.")

conn = sqlite3.connect(DB_FILE)
conn.execute("PRAGMA foreign_keys = ON;")

load_robots(conn)
load_intervals(conn)

for idx, fname in enumerate(TRAJ_FILES, start=1):
    if not os.path.exists(fname):
        raise SystemExit(f"Error: trajectory file '{fname}' not found. Upload it first.")
    load_trajectory(conn, fname, idx)

conn.close()
print("All data loaded successfully.")


Loaded robots.
Loaded target intervals.
Loaded t1.csv → Robot 1
Loaded t2.csv → Robot 2
Loaded t3.csv → Robot 3
Loaded t4.csv → Robot 4
Loaded t5.csv → Robot 5
All data loaded successfully.


In [ ]:
import sqlite3
import pandas as pd

# Connect to the database
conn = sqlite3.connect("robot.db")

# List of expected table names
tables = ["Robot", "TargetInterval", "SensorReading"]

# Display each table using pandas
for table in tables:
    print(f"\n Table: {table}")
    df = pd.read_sql_query(f"SELECT * FROM {table}", conn)
    display(df)

conn.close()




 Table: Robot


,robot_id,name
0,1,Astro
1,2,IamHuman
2,3,MoonLander
3,4,Wonderlust
4,5,Challenger



 Table: TargetInterval


,interval_id,start_time_sec,end_time_sec,event_type
0,1,5000,5300,unknown
1,2,3000,3800,unknown
2,3,6000,6500,normal



 Table: SensorReading


,robot_id,timestamp,x_cm,y_cm,interval_id
0,1,1,145.00,130.00,NaN
1,1,2,145.00,130.00,NaN
2,1,3,145.00,130.00,NaN
3,1,4,145.00,130.00,NaN
4,1,5,145.00,130.00,NaN
...,...,...,...,...,...
47995,5,9596,371.82,291.98,NaN
47996,5,9597,371.82,291.98,NaN
47997,5,9598,371.63,291.72,NaN
47998,5,9599,371.33,291.32,NaN


# Task 3: (5 pt) Using SQL, return the following information related to meta-info of the data (print out the query result is sufficient).

 1. A table consists of the names of robots and the maximal x-axis, minimum
 x-axis reached by this robot.


In [ ]:
import sqlite3

conn = sqlite3.connect("robot.db")
cur = conn.cursor()

statement = '''
SELECT R.name AS name, MAX(S.x_cm) AS max_x, MIN(S.x_cm) AS min_x
FROM Robot R JOIN SensorReading S ON R.robot_id = S.robot_id
GROUP BY R.robot_id
'''

res = cur.execute(statement)

relational_schema = [x[0] for x in res.description]
print(relational_schema)
print('------------')
for x in res:
    print(x)

conn.close()

['name', 'max_x', 'min_x']
------------
('Astro', 145.0, -4.7466)
('IamHuman', 233.38, -6.8667)
('MoonLander', 145.0, -190.85)
('Wonderlust', 380.8, -380.93)
('Challenger', 382.31, -192.93)


 2. A table consists of the names of robots and the maximal y-axis, minimum
 y-axis reached by this robot.

In [ ]:
import sqlite3

conn = sqlite3.connect("robot.db")
cur = conn.cursor()

statement = '''
SELECT R.name AS name, MAX(S.y_cm) AS max_y, MIN(S.y_cm) AS min_y
FROM Robot R JOIN SensorReading S ON R.robot_id = S.robot_id
GROUP BY R.robot_id
'''

res = cur.execute(statement)

relational_schema = [x[0] for x in res.description]
print(relational_schema)
print('------------')
for x in res:
    print(x)

conn.close()

['name', 'max_y', 'min_y']
------------
('Astro', 175.8, 0.41976)
('IamHuman', 195.1, -0.72259)
('MoonLander', 163.13, 6.9552)
('Wonderlust', 130.0, -380.87)
('Challenger', 382.67, -380.72)


#Task 4: (5 pt) Using SQL, write code to analyze the following info related to robot trajectory:

 1. Suppose we define two robots are close with each other if ‘both x-axis and
 y-axis’ difference is smaller than 1 cm. Return all the regions (measured
 by x min, x max, y min, y max) that robot “Astro” and “IamHuman”
 are close with each other.

In [ ]:
import sqlite3

conn = sqlite3.connect("robot.db")
cur = conn.cursor()

statement = '''
SELECT MIN(MIN(A.x_cm, B.x_cm)) AS x_min, MAX(MAX(A.x_cm, B.x_cm)) AS x_max, MIN(MIN(A.y_cm, B.y_cm)) AS y_min, MAX(MAX(A.y_cm, B.y_cm)) AS y_max
FROM SensorReading A JOIN Robot RA ON A.robot_id = RA.robot_id JOIN SensorReading B ON A.timestamp = B.timestamp JOIN Robot RB ON B.robot_id = RB.robot_id
WHERE RA.name = "Astro" AND RB.name = "IamHuman"
AND ABS(A.x_cm - B.x_cm) < 1
AND ABS(A.y_cm - B.y_cm) < 1
'''

res = cur.execute(statement)

relational_schema = [x[0] for x in res.description]
print(relational_schema)
print('------------')
for x in res:
    print(x)

conn.close()

['x_min', 'x_max', 'y_min', 'y_max']
------------
(145.0, 145.65, 130.0, 130.52)


 2. For the same robots, measured how many secs that they are close with
 each other.

In [ ]:
import sqlite3

conn = sqlite3.connect("robot.db")
cur = conn.cursor()

statement = '''
SELECT COUNT(*) as num_seconds_close
FROM SensorReading A JOIN Robot RA ON A.robot_id = RA.robot_id JOIN SensorReading B ON A.timestamp = B.timestamp JOIN Robot RB ON B.robot_id = RB.robot_id
WHERE RA.name = "Astro" AND RB.name = "IamHuman"
AND ABS(A.x_cm - B.x_cm) < 1
AND ABS(A.y_cm - B.y_cm) < 1
ORDER BY A.timestamp
'''

res = cur.execute(statement)

relational_schema = [x[0] for x in res.description]
print(relational_schema)
print('------------')
for x in res:
    print("This is how long they were together close together for (seconds):",x[0])

conn.close()

['num_seconds_close']
------------
This is how long they were together close together for (seconds): 42


 3. (bonus) For all the target interval, calculate if the average robot moving
 speed is smaller than 0.2 cm/s. You code should print out a table that
 each row consists the interval id, and the answer (Yes or No). Feel free to
 use python to print out this table

In [ ]:
import sqlite3
conn = sqlite3.connect("robot.db")
cur = conn.cursor()

statement = '''
SELECT
    interval_id,
    ROUND(AVG(speed), 4) AS avg_speed_cm_s,
    CASE
        WHEN AVG(speed) < 0.2 THEN 'Yes'
        ELSE 'No'
    END AS Is_Speed_Less_Than_0_2
FROM (
    SELECT
        interval_id,
        robot_id,
        SQRT(
            (MAX(x_cm) - MIN(x_cm)) * (MAX(x_cm) - MIN(x_cm)) +
            (MAX(y_cm) - MIN(y_cm)) * (MAX(y_cm) - MIN(y_cm))
        ) /
        CASE
            WHEN MAX(timestamp) - MIN(timestamp) = 0 THEN 1
            ELSE MAX(timestamp) - MIN(timestamp)
        END AS speed
    FROM SensorReading
    WHERE interval_id IS NOT NULL
    GROUP BY interval_id, robot_id
)
GROUP BY interval_id;

'''

res = cur.execute(statement)

relational_schema = [x[0] for x in res.description]
print(relational_schema)
print('------------')
for x in res:
    print(x)

conn.close()

['interval_id', 'avg_speed_cm_s', 'Is_Speed_Less_Than_0_2']
------------
(1, 0.2001, 'No')
(2, 0.2765, 'No')
(3, 0.2833, 'No')
